<a href="https://colab.research.google.com/github/shretimag/eigen_portfolio/blob/main/PCA_Portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip -q install yfinance

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [ ]:
name=['GOOG','AMZN','SBIN.BO','MSFT','LT.NS','RELIANCE.NS','TATASTEEL.NS','HDFCLIFE.NS','ITC.NS','MARUTI.NS','COALINDIA.NS','BHARTIARTL.NS','NTPC.NS','TATACONSUM.NS','KOTAKBANK.NS','NESTLEIND.NS','BRITANNIA.NS','BAJFINANCE.NS','BAJAJFINSV.NS','RELIANCE.NS']

In [ ]:
def dataset(name):
  y_f = pd.DataFrame() #empty dataframe
  for i in name:
    y = yf.Ticker(i)
    d = y.history(interval = "1d", start="2024-01-01", end='2024-03-01')
    n_d = (d['Close']-d['Open'])*100/d['Open']
    list_ = (d.index).tolist()
    n_d_n = n_d.to_numpy().reshape(1,-1)
    df = pd.DataFrame(n_d_n)
    y_f = y_f.append(df, ignore_index = True)
  return y_f.T, list_

In [ ]:
y_f , list_ = dataset(name)
y_f.columns = name

In [ ]:
y_f

In [ ]:
y_f = y_f.assign(Date = list_)
y_f = y_f.set_index(keys='Date') # y_f.set_index(keys='Date', inplace = True)
y_f.head()

In [ ]:
asset_returns = pd.DataFrame(data=np.zeros(shape=(len(y_f.index), y_f.shape[1])),
                             columns=y_f.columns.values,
                             index=y_f.index)
normed_returns = asset_returns
# normed_returns is pandas.DataFrame that should contain normalized returns
asset_returns = y_f.pct_change(1).dropna()
normed_returns = (asset_returns - asset_returns.mean()) / (asset_returns.std() + 1e-8) # (x-mu)/std
normed_returns = normed_returns.dropna(axis = 1)# removing null/nan values

normed_returns.head()

In [ ]:
normed_returns.shape

In [ ]:
train_end = "2024-02-15" #train_data = [1 Jan- 15 Jan] test = [16 Jan - 1 Mar]

df_train = None
df_test = None
df_raw_train = None
df_raw_test = None

df_train = normed_returns[normed_returns.index <= train_end].copy()
df_test = normed_returns[normed_returns.index > train_end].copy()

df_raw_train = asset_returns[asset_returns.index <= train_end].copy()
df_raw_test = asset_returns[asset_returns.index > train_end].copy()

print('Train dataset:', df_train.shape)
print('Test dataset:', df_test.shape)

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components = 7)

In [ ]:
fit = pca.fit_transform(df_train)
df_pca = pd.DataFrame(fit,columns=np.arange(pca.n_components_))
df_pca.head()

In [ ]:
df_pca.shape

In [ ]:
def sharpe_ratio(ts_returns, periods_per_year=252):
    annualized_return = 0.
    annualized_vol = 0.
    annualized_sharpe = 0.

    n_years = ts_returns.shape[0] / periods_per_year
    annualized_return = np.power(np.prod(1 + ts_returns),(1 / n_years)) - 1
    annualized_vol = ts_returns.std() * np.sqrt(periods_per_year)
    annualized_sharpe = annualized_return / annualized_vol


    return annualized_return, annualized_vol, annualized_sharpe